In [1]:
from PIL import Image
import torch
import clip
import pandas as pd
from tqdm import tqdm
import os

/home/raph/fast_mem/anaconda3/envs/BeautyPredict/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
emb_dir = "embeddings"

In [38]:
# Загрузка предобученной модели CLIP
# name = "RN50"
# name = "RN50x4"
# name = "RN101"
# name = "ViT-B/32"
# name = "ViT-B/16"
name = "ViT-L/14"
model, preprocess = clip.load(name, device=device)
name = name.replace("/", "")

In [39]:
full_set = pd.read_csv("full_set.csv")

In [40]:
# сохранение всех эмбеддингов
ids = []
img_embs = []
text_embs = []

img_emb_df = pd.DataFrame()
text_emb_df = pd.DataFrame()
both_emb_df = pd.DataFrame()


for index, row in tqdm(full_set.iterrows()):
    img = preprocess(Image.open(row["img_path"])).unsqueeze(0).to(device)
    promt = clip.tokenize(row["img_generation_promt"]).to(device)
    
    with torch.no_grad():
        img_emb = model.encode_image(img)
        text_emb = model.encode_text(promt)
        both_emb = torch.cat([img_emb, text_emb], axis=1)

    img_column_names = [f'emb{i}' for i in range(img_emb.size(1))]
    img_tmp_df = pd.DataFrame(img_emb.cpu().numpy(), columns=img_column_names, dtype="float16")
    img_tmp_df["id"] = row["id"]
    img_emb_df = pd.concat([img_emb_df, img_tmp_df])
    
    
    text_column_names = [f'emb{i}' for i in range(text_emb.size(1))]
    text_tmp_df = pd.DataFrame(text_emb.cpu().numpy(), columns=text_column_names, dtype="float16")
    text_tmp_df["id"] = row["id"]
    text_emb_df = pd.concat([text_emb_df, text_tmp_df])
    
    both_column_names = [f'emb{i}' for i in range(both_emb.size(1))]
    both_tmp_df = pd.DataFrame(both_emb.cpu().numpy(), columns=both_column_names, dtype="float16")
    both_tmp_df["id"] = row["id"]
    both_emb_df = pd.concat([both_emb_df, both_tmp_df])

    
img_emb_df = img_emb_df[["id", *img_column_names]]
img_emb_df.to_csv(os.path.join(emb_dir, name + "_img.csv"), index=False)

text_emb_df = text_emb_df[["id", *text_column_names]]
text_emb_df.to_csv(os.path.join(emb_dir, name + "_text.csv"), index=False)

both_emb_df = both_emb_df[["id", *both_column_names]]
both_emb_df.to_csv(os.path.join(emb_dir, name + "_both.csv"), index=False)


2256it [08:34,  4.38it/s]


In [32]:
check_df = pd.read_csv(os.path.join(emb_dir, name + "_img.csv"))
(text_emb.cpu() == torch.tensor(check_df.drop("id", axis=1).values, dtype=torch.float16)).any()

tensor(True)

In [33]:
check_df = pd.read_csv(os.path.join(emb_dir, name + "_text.csv"))
(text_emb.cpu() == torch.tensor(check_df.drop("id", axis=1).values, dtype=torch.float16)).any()

tensor(True)

In [34]:
check_df = pd.read_csv(os.path.join(emb_dir, name + "_both.csv"))
(both_emb.cpu() == torch.tensor(check_df.drop("id", axis=1).values, dtype=torch.float16)).any()

tensor(True)

In [22]:
torch.tensor(check_df.iloc[-1].drop("id").values, dtype=torch.float16).to(torch.float32)

tensor([-0.0165, -0.0292, -0.0511,  ..., -0.3103,  0.0547, -0.1815])